In [1]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, TaskType

In [3]:
df = pd.read_csv("shortjokes.csv")
df.head()

,ID,Joke
0,1,"[me narrating a documentary about narrators] ""..."
1,2,Telling my daughter garlic is good for you. Go...
2,3,I've been going through a really rough period ...
3,4,"If I could have dinner with anyone, dead or al..."
4,5,Two guys walk into a bar. The third guy ducks.


In [ ]:
# Preprocessing data

df = df.dropna(subset = ["Joke"])
df = df[df["Joke"].apply(lambda x: isinstance(x,str) and len(x.strip()) > 10)]
df.head()

,ID,Joke
0,1,"[me narrating a documentary about narrators] ""..."
1,2,Telling my daughter garlic is good for you. Go...
2,3,I've been going through a really rough period ...
3,4,"If I could have dinner with anyone, dead or al..."
4,5,Two guys walk into a bar. The third guy ducks.


In [ ]:
# Get only 1000 samples
df = df.sample(1000, random_state=42).reset_index(drop=True)

In [6]:
dataset = Dataset.from_pandas(df[["Joke"]].rename(columns={"Joke": "text"}))
dataset

Dataset({
    features: ['text'],
    num_rows: 1000
})

In [ ]:
# Load the model and tokenizer
model_name ="gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# Tokenizing definition
def tokenize(example):
  encoding = tokenizer(example["text"], truncation = True, return_tensors = "pt",max_length = 128, padding = "max_length")
  item = {}
  for k,v in encoding.items():
    item[k] = v.squeeze(0)
  item["labels"] = item["input_ids"].clone()        # For training inputs = labels
  return item

In [ ]:
# Map the tokenizer to the dataset
tokenized_dataset = dataset.map(tokenize)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(model_name)
base_model.resize_token_embeddings(len(tokenizer))

In [ ]:
# Configure LoRA parameters

lora_config = LoraConfig(
    r=8,                                    # LoRA Rank
    lora_alpha=16,                          # LoRA Alpha
    target_modules=["c_attn", "c_proj"],    # LoRA Target Modules
    lora_dropout=0.05,                      # LoRA Dropout
    bias="none",                            # LoRA Bias
    task_type=TaskType.CAUSAL_LM
)

In [12]:
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

trainable params: 811,008 || all params: 125,250,816 || trainable%: 0.6475


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1768: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [ ]:
# Training Arguments to be attached to Trainer
training_args = TrainingArguments(
    output_dir="./gpt2-jokes-lora",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    fp16=True,
    report_to="none",
    push_to_hub=False,
)

In [ ]:
# Configuring Trainer Parameters
trainer = Trainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
# Train the model
trainer.train()
model.save_pretrained("gpt2-jokes-lora")
tokenizer.save_pretrained("gpt2-jokes-lora")

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,4.522200
20,4.406700
30,4.253700
40,4.365500
50,4.353900
60,4.270200
70,4.209600
80,4.137500
90,4.139100
100,3.861900


('gpt2-jokes-lora/tokenizer_config.json',
 'gpt2-jokes-lora/special_tokens_map.json',
 'gpt2-jokes-lora/vocab.json',
 'gpt2-jokes-lora/merges.txt',
 'gpt2-jokes-lora/added_tokens.json',
 'gpt2-jokes-lora/tokenizer.json')

In [ ]:
# Inference

from transformers import pipeline

pipe = pipeline("text-generation", model="gpt2-jokes-lora", tokenizer = tokenizer)

prompt = "Why did the chicken cross the road?"
print(pipe(prompt, max_length=10, top_k=25, top_p=0.95)[0]["generated_text"])


Device set to use cuda:0
Both `max_new_tokens` (=256) and `max_length`(=10) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Why did the chicken cross the road? It was the same chicken the guy had the other day. The chicken was in the wrong place at the wrong time. He just ate it and didn't know where it was. A friend suggested he go back to the mall and buy some more chicken. The chicken didn't even go back. It was the same chicken the guy had the other day. The chicken was in the wrong place at the wrong time. He just ate it and didn't know where it was. A friend suggested he go back to the mall and buy some more chicken. The chicken didn't even go back. It was the same chicken the guy had the other day. The chicken was in the wrong place at the wrong time. He just ate it and didn't know where it was. A friend suggested he go back to the mall and buy some more chicken. The chicken didn't even go back. It was the same chicken the guy had the other day. The chicken was in the wrong place at the wrong time. He just ate it and didn't know where it was. A friend suggested he go back to the mall and buy some mor

### Quantising my Model with GPTQ

In [22]:
! pip install auto_gptq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 104.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu1

In [ ]:
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
from transformers import AutoTokenizer

model_path = "gpt2-jokes-lora"

quantize_config = BaseQuantizeConfig(           # Qunatisation with GPTQ
    bits=4,                                     # Final values to be represented in 4 bits
    group_size=128,                             
    desc_act=False,
)

tokenizer = AutoTokenizer.from_pretrained(model_path)

quant_model = AutoGPTQForCausalLM.from_pretrained(
    model_path,
    quantize_config=quantize_config,
    trust_remote_code=True,
)

inputs = tokenizer("Hello, this is a test input for GPTQ.", return_tensors="pt")
quant_model.quantize([inputs])                   # Quantisation happens here

# Save Model and Tokenizer
quant_model.save_quantized("gpt2-jokes-gptq")
tokenizer.save_pretrained("gpt2-jokes-gptq")

/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:410: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:418: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:461: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float16)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `

('gpt2-jokes-gptq/tokenizer_config.json',
 'gpt2-jokes-gptq/special_tokens_map.json',
 'gpt2-jokes-gptq/vocab.json',
 'gpt2-jokes-gptq/merges.txt',
 'gpt2-jokes-gptq/added_tokens.json',
 'gpt2-jokes-gptq/tokenizer.json')

In [ ]:
# Inference with Quantised Model

import torch
from transformers import AutoTokenizer
from auto_gptq import AutoGPTQForCausalLM

model_path = "gpt2-jokes-gptq/gpt2-jokes-gptq"

tokenizer = AutoTokenizer.from_pretrained(model_path)

model = AutoGPTQForCausalLM.from_quantized(
    model_path,
    device="cuda:0",
    use_triton=False,
    trust_remote_code=True,
)

prompt = "Why did the chicken cross the road?"

input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda:0")

generated_ids = input_ids
for _ in range(30):
    with torch.no_grad():
        outputs = model(generated_ids)
        logits = outputs.logits[:, -1, :]
        probabilities = torch.softmax(logits, dim=-1)
        next_token_id = torch.argmax(probabilities, dim=-1).unsqueeze(0)

        generated_ids = torch.cat([generated_ids, next_token_id], dim=-1)

generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

print("Prompt:", prompt)
print("Generated:", generated_text)